In [1]:
import matplotlib.pyplot as plt
import numpy as np

from galaxy_tools import load_halo, load_sim, mask_sim_to_halo, get_mock_observation
from general_tools import radius_of_param_limit, mag_to_lum_SB, measure_surfbright

from firestudio.utils.stellar_utils import raytrace_projection

%matplotlib inline


Couldn't import numba. Missing:
abg_python.all_utils.get_cylindrical_velocities
Missing firefly, obtain it at http://github.com/ageller/Firefly or pip install firefly.


In [2]:
sim = 'm12_elvis_RomulusRemus_res4000'

snap_num = 600

# path to the directory holding the simulation snapshot 
sim_path =  '/data17/grenache/aalazar/FIRE/GVB/'+str(sim)+'/output/hdf5/snapdir_'+str(snap_num)+'/'

# path to the directory holding the halo files for the simulation
halo_path = '/data17/grenache/aalazar/FIRE/GVB/'+str(sim)+'/halo/rockstar_dm/hdf5/'


In [3]:
# This loads the halo file
# host=True means it only returns the host halo information, otherwise it returns all the halos
# filetype='hdf5' indicate the halo file type, some halos are filetype='ascii'
# hostnumber indicates which host halo you are looking at. Only need to worry about this for the elvis pairs.
        #use hostnumber=2 to get the halo info for the second host halo. 
        #ex: halo_themla = load_halo(halo_path, snap_num, host=True, filetype='hdf5', hostnumber=1)
        #    halo_louise = load_halo(halo_path, snap_num, host=True, filetype='hdf5', hostnumber=2)


halo = load_halo(halo_path, snap_num, host=True, filetype='hdf5', hostnumber=2)


In [4]:
# This loads the star and gas particle
# It only loads the parameters I need for the mock images

star_snapdict, gas_snapdict = load_sim(sim_path,snap_num)


KeyboardInterrupt: 

In [ ]:
# This centers star and gas dictionaries on the host halo
# If orient=True, then it will rotate the particles based on the disk orientation
# If lim=True, then it masks out the particles beyond the limvalue
    # Have lim=False if you want to carry around all the particles
    # I chose to mask out particles beyond the virial radius

limvalue = halo['Halfmass_Radius'].values[0]/3

star_snapdict, gas_snapdict, halo = mask_sim_to_halo(star_snapdict=star_snapdict,
                                                         gas_snapdict=gas_snapdict, 
                                                         host_halo=halo, 
                                                         orient=True,
                                                         lim = True,
                                                         limvalue = limvalue
                                                        )


In [ ]:
plt.hexbin(star_snapdict['Coordinates'][:,0],star_snapdict['Coordinates'][:,1], bins='log')
plt.gca().set_aspect(1)

In [ ]:
# mass_scaler: The code expects the masses to be in the simulation units of 1e10 Msun.
              #if the mass is in units Msun (which it is for the snapdict), 
              #then use mass_scaler = 1e-10
        
# bands: The code will determine the luminosity in 3 bands because the main goal of this code
        #is to make the a composite image. 
        #Band options are:     
        #    0  - Absolute mag
        #    1  - SDSS u (unprimed AB)
        #    2  - SDSS g (unprimed AB)
        #    3  - SDSS r (unprimed AB)
        #    4  - SDSS i (unprimed AB)
        #    5  - SDSS z (unprimed AB)
        #    6  - U (BESSEL)
        #    7  - B (BESSEL)
        #    8  - V (BESSEL)
        #    9  - R (KPNO)
        #    10 - I (KPNO)
        #    11 - J (BESSEL)
        #    12 - H (BESSEL)
        #    13 - K (BESSEL)
        #For my research I usually use u, g, r

mass_scaler = 1e-10
bands = [1,2,3]


#lums is the luminosity of each particle in units of 1e-10 Lsum
#kappa has to do with the opacity of the band

kappas,lums = raytrace_projection.read_band_lums_from_tables([1,2,3],
                                                             star_snapdict['Masses']*mass_scaler,
                                                             star_snapdict['StellarAge'],
                                                             star_snapdict['Metallicity'])


# The lums output units are in 1e-10 Lsum
unit_factor = 1e10 # gives units in Lsun
lums[0], lums[1], lums[2] = lums[0]*unit_factor, lums[1]*unit_factor, lums[2]*unit_factor



In [ ]:
# To get the Half light radius use radius_of_param_limit
# For more information use radius_of_param_limit?

r_50_im, l_50_im = radius_of_param_limit(radii=star_snapdict['r_xy'], 
                                       parameter=lums[1], 
                                       limits=[.5], 
                                       limit_type='param_fraction')

print('Half light radius determined from the g-band stellar luminosity')
print(f'The half light radius is: {r_50_im[0]:.2f} kpc')
print(f'The luminosity contained within the half light radius is: {l_50_im[0]/1e10:.2f} Lsun/1e10')
   

In [ ]:
# If you also want the mock image use this method
# The output of thus function is the composite image, 
# and the array of the surface brightness in each band 

#Field of view for the image in kpc
FOV = limvalue
#Number 0f pixels
pixels= 100
# the axis it is projected on, 'xy', 'yz', 'zx'
view = 'xy' 
# use the center of mass or center of light, 'light', 'mass'
center = 'light'
# the limiting surface brightness, if you dont want to limit it set it equal to zero
minden = mag_to_lum_SB(29,None) # I set it to 29 mag/arcsec^2. This function convers it to lum/kpc^2


image,out_band0,out_band1,out_band2 = get_mock_observation(star_snapdict,
                                                               gas_snapdict,
                                                               bands = [1,2,3],
                                                               FOV=FOV,
                                                               mass_scaler = 1e-10,
                                                               pixels=pixels,
                                                               view='xy',
                                                               minden = minden,
                                                               center = center
                                                            )

In [ ]:
plt.imshow(image)

In [ ]:
radius,lum_in_shell = measure_surfbright(out_band1,FOV,return_type='shell_lum')

r_50_lum, l_50_lum = radius_of_param_limit(radii=radius,
                                               parameter=lum_in_shell, 
                                               limits=[.5],
                                               limit_type='param_fraction')

print('Half Light radius from mock g-band Image')
print(f'The half light radius is: {r_50_lum[0]:.2f} kpc')
print(f'The luminosity contained within the half light radius is: {l_50_lum[0]/1e10:.2f} Lsun/1e10')

In [ ]:
sum_light = np.zeros(len(radius))

for i in range(len(radius)):
    rmask = (star_snapdict['r_xy'] <  radius[i])
    sum_light[i] = np.sum(lums[1][rmask]) 


In [ ]:
plt.plot(radius,sum_light/sum_light[-1],label='Luminosity')
plt.plot(radius,np.cumsum(lum_in_shell)/np.cumsum(lum_in_shell)[-1],label='Image')
plt.gca().set_xscale('log')
plt.axhline(y=0.5, color='k', linestyle='-', label='Half Light')
plt.axvline(x=r_50_lum[0], color='r', linestyle='--')
plt.axvline(x=r_50_im[0], color='r', linestyle='--')
plt.title('Fraction of Light')
plt.xlabel('Radius [kpc]')
plt.legend()